- To start server run: `python app.py` 
- To start server + Prometheus + Grafana run: `docker-compose up --build`

In [1]:
import sys
import os

# Get the absolute path of the app root
app_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the neighbor directory to sys.path
sys.path.append(app_root)

In [2]:
# Necessary imports for gRPC
import grpc
from grpc_protobuf.model_manager_grpc_pb2_grpc import ModelManagerGRPCStub
from grpc_protobuf.model_manager_grpc_pb2 import FloatList, FitRequest, PredictRequest, SeedRequest, SelectExperimentRequest, PredictResponse, BasicExperimentResponse, BasicSuccessResponse, ExperimentMetadataResponse, ListExperimentsResponse
from google.protobuf.empty_pb2 import Empty

In [3]:

# Create a gRPC channel and stub
HOST_GRPC = 'localhost'
PORT_GRPC = 50051  # Change this to your server's port
channel = grpc.insecure_channel(f'{HOST_GRPC}:{PORT_GRPC}')
stub = ModelManagerGRPCStub(channel)


In [4]:
# Example request to list experiments
def list_experiments():
    request = Empty()
    response = stub.ListExperiments(request)
    return response


In [5]:
def select_experiment(experiment_id):
    experiment_id = str(experiment_id)
    request = SelectExperimentRequest(experiment_id=experiment_id)
    response = stub.SelectExperiment(request)
    return response

In [6]:
# Example request to fit
def fit(X_train, y_train):
    X_train = [FloatList(values=sample) for sample in X_train]
    request = FitRequest(X_train=X_train, y_train=y_train)
    response = stub.Fit(request)
    return response

In [7]:
# Example request to predict
def predict(X_test):
    X_test = [FloatList(values=sample) for sample in X_test]
    request = PredictRequest(X_test=X_test)
    response = stub.Predict(request)
    return response


In [8]:
# Example request to seed
def seed(value):
    request = SeedRequest(value=value)
    response = stub.Seed(request)
    return response

In [9]:
msg = seed(42)
print(msg)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:50051: Failed to connect to remote host: connect: Connection refused (61)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-11-12T03:20:51.912247+03:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:50051: Failed to connect to remote host: connect: Connection refused (61)"}"
>

In [10]:
# Call the functions and print the results
experiments = list_experiments()
print("Experiments:", experiments)

Experiments: experiments {
  experiment_id: "9cfbb98a-2d85-4a71-8d90-c1b308d739b9"
  created_dttm: "2024-11-06T13:38:57.734007"
  last_changed_dttm: "2024-11-06T13:38:57.734007"
  model_filename: "model.skl"
  name: "Template untrained DecisionTreeClassifier Scikit-Learn model"
  template_flg: true
}
experiments {
  experiment_id: "3a71e9aa-40cd-4ea6-a41b-0476ded16a4c"
  created_dttm: "2024-11-06T13:38:57.732099"
  last_changed_dttm: "2024-11-06T13:38:57.732099"
  model_filename: "model.pt"
  name: "Template untrained MLP PyTorch model"
  template_flg: true
}
experiments {
  experiment_id: "c2d945f2-946e-4112-8d68-077b52d02528"
  created_dttm: "2024-11-06T13:38:57.733386"
  last_changed_dttm: "2024-11-06T13:38:57.733386"
  model_filename: "model.skl"
  name: "Template untrained LinearRegression Scikit-Learn model"
  template_flg: true
}



In [11]:
response = select_experiment(experiment_id='c2d945f2-946e-4112-8d68-077b52d02528')
print(response)

success: true



In [12]:
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Generate sample data
# Number of samples
n_samples = 20
n_features = 5

# Independent variable (X)
X_train = [np.random.uniform(0, 10, size=n_features).tolist() for obs in range(n_samples)]

# True parameters
slopes = np.random.uniform(0, 10, size=n_features).tolist()
intercept = 1.0

# Generate dependent variable (Y) with some noise
noise = np.random.normal(0, 1, n_samples).tolist()  # Gaussian noise
y_train = [(sum(np.array(slopes) * np.array(sample)) + intercept + err) for sample, err in zip(X_train, noise)]

# Display the samples
print("Sample Data X:")
for observation in X_train:
    print(list(observation), ',')
print()
print("Sample Data y:")
for observation in y_train:
    print(observation, ',')



Sample Data X:
[3.745401188473625, 9.50714306409916, 7.319939418114051, 5.986584841970366, 1.5601864044243652] ,
[1.5599452033620265, 0.5808361216819946, 8.661761457749352, 6.011150117432088, 7.080725777960454] ,
[0.20584494295802447, 9.699098521619943, 8.324426408004218, 2.1233911067827616, 1.8182496720710062] ,
[1.8340450985343382, 3.0424224295953772, 5.247564316322379, 4.319450186421157, 2.9122914019804194] ,
[6.118528947223795, 1.3949386065204183, 2.9214464853521815, 3.663618432936917, 4.56069984217036] ,
[7.851759613930136, 1.9967378215835974, 5.142344384136116, 5.924145688620425, 0.46450412719997725] ,
[6.075448519014383, 1.7052412368729153, 0.6505159298527952, 9.488855372533333, 9.656320330745594] ,
[8.08397348116461, 3.0461376917337066, 0.9767211400638387, 6.842330265121569, 4.4015249373960135] ,
[1.2203823484477883, 4.951769101112702, 0.34388521115218396, 9.093204020787821, 2.587799816000169] ,
[6.62522284353982, 3.1171107608941098, 5.200680211778108, 5.4671027934327965, 1.848

In [13]:
response = fit(X_train, y_train)
print(response)

success: true



In [14]:
import numpy as np

# Generate test data
# Number of samples
n_samples_test = 10

# Independent variable (X)
X_test = [np.random.uniform(0, 10, size=n_features).tolist() for obs in range(n_samples_test)]
# Display the samples
print("Test Data X:")
for observation in X_test:
    print(list(observation), ',')

Test Data X:
[2.2210781047073027, 1.198653673336828, 3.3761517140362796, 9.429097039125192, 3.2320293202075523] ,
[5.187906217433661, 7.030189588951778, 3.63629602379294, 9.717820827209607, 9.624472949421111] ,
[2.5178229582536416, 4.972485058923855, 3.0087830981676964, 2.848404943774676, 0.36886947354532795] ,
[6.095643339798968, 5.026790232288615, 0.5147875124998935, 2.7864646423661146, 9.082658859666537] ,
[2.395618906669724, 1.448948720912231, 4.89452760277563, 9.856504541106007, 2.4205527151150044] ,
[6.721355474058786, 7.616196153287175, 2.3763754399239967, 7.282163486118596, 3.677831327192532] ,
[6.3230583059357945, 6.335297107608947, 5.357746840747585, 0.902897700544083, 8.35302495589238] ,
[3.2078006497173583, 1.8651851039985423, 0.40775141554763916, 5.908929431882418, 6.775643618422825] ,
[0.16587828927856152, 5.12093058299281, 2.2649577519793795, 6.451727904094499, 1.7436642900499144] ,
[6.90937738102466, 3.867353463005374, 9.367299887367345, 1.3752094414599325, 3.4106635105

In [15]:
response = predict(X_test)
print(response)

predictions: 96.7044296
predictions: 195.462402
predictions: 60.3536
predictions: 133.289963
predictions: 97.9369278
predictions: 129.776276
predictions: 140.641785
predictions: 106.387421
predictions: 88.5956039
predictions: 95.3442841

